In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# SubproblemComp

`SubproblemComp` provides a way to evaluate an OpenMDAO system within an OpenMDAO system.

# SubproblemComp Options

In [ ]:
import openmdao.api as om

submodel1 = om.Group()
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

om.show_options_table(om.SubproblemComp(model=submodel1, inputs=['r', 'theta'],
                          outputs=['x']))

# SubproblemComp Constructor

The call signature for the `SubproblemComp` constructor is

```{eval-rst}
    .. automethod:: openmdao.components.subproblem_comp.SubproblemComp.__init__
        :noindex:
```

# Using the SubproblemComp

`SubproblemComp` allows you to add a component that is a `System` itself. Its required arguments are `model`, `inputs`, and `outputs`. Other optional arguments are arguments you can use to configure the `Problem` within the subproblem (see [problem.py](../../../_srcdocs/packages/core/problem.html#openmdao.core.problem.Problem)). The only difference is, the kwargs used for `Problem` have to be passed into `SubproblemComp` as a `dict` rather than as kwargs.

The arg that stores the kwargs for `Problem` is `prob_options`. Other kwargs for `ExplicitComponent` can then be passed normally to `SubproblemComp`.

`SubproblemComp` provides a way to evaluate an OpenMDAO system within an OpenMDAO system. Unlike a typical sub-system, using a subproblem exposes only a limited number of inputs and outputs of the underlying model to the parent system. This may be beneficial from a performance standpoint when there are many inputs and outputs of the internal system that aren't needed in the external system.

Note that this means that nonlinear and linear solvers are needed on the subproblem's model if implicit behavior is present, since the internal variables are invisible to solvers in the parent system.

At this time, OpenMDAO does not evaluate gradients through the optimization process, so a gradient-based driver at the outer level cannot access derivatives across the optimization on the inner level.

## IO Format

Inputs and outputs can be passed into `SubproblemComp` as a list of `str`, `tuple`, or both. If the list element is `str`, then it must be the group level promoted name that is passed. For this, the IO to be referred to must be promoted in `add_subsystem`. If the list element is `tuple`, then the first element of the `tuple` must be the group level promoted name or the absolute name of the variable, and the second element is the alias that `SubproblemComp` will refer to it as.

e.g. (path.to.var, desired_var_name)  
or (var, desired_var_name) if the variable is already promoted at the group level

Input and output names can be wildcards too, only if their type is `str`.


# Example: Implementing SubproblemComp

In [ ]:
import openmdao.api as om
from numpy import pi

p = om.Problem()

model = om.Group()
model.add_subsystem('supComp', om.ExecComp('z = x**2 + y'),
                    promotes_inputs=['x', 'y'],
                    promotes_outputs=['z'])

submodel1 = om.Group()
submodel1.add_subsystem('sub1_ivc_r', om.IndepVarComp('r', 1.),
                        promotes_outputs=['r'])
submodel1.add_subsystem('sub1_ivc_theta', om.IndepVarComp('theta', pi),
                        promotes_outputs=['theta'])
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

submodel2 = om.Group()
submodel2.add_subsystem('sub2_ivc_r', om.IndepVarComp('r', 2),
                        promotes_outputs=['r'])
submodel2.add_subsystem('sub2_ivc_theta', om.IndepVarComp('theta', pi/2),
                        promotes_outputs=['theta'])
submodel2.add_subsystem('subComp2', om.ExecComp('y = r*sin(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['y'])

subprob1 = om.SubproblemComp(model=submodel1, inputs=['r', 'theta'],
                            outputs=['x'])
subprob2 = om.SubproblemComp(model=submodel2, inputs=['r', 'theta'],
                            outputs=['y'])

p.model.add_subsystem('sub1', subprob1, promotes_inputs=['r','theta'],
                            promotes_outputs=['x'])
p.model.add_subsystem('sub2', subprob2, promotes_inputs=['r','theta'],
                            promotes_outputs=['y'])
p.model.add_subsystem('supModel', model, promotes_inputs=['x','y'],
                            promotes_outputs=['z'])

p.setup(force_alloc_complex=True)

p.set_val('r', 1)
p.set_val('theta', pi)

p.run_model()
cpd = p.check_partials(method='cs', out_stream=None)

# Example: Variable Aliases

In [ ]:
import openmdao.api as om

p = om.Problem()
model = om.Group()

model.add_subsystem('subsys', om.ExecComp('z = x**2 + y**2'))
subprob = om.SubproblemComp(model=model, inputs=[('subsys.x', 'a'), ('subsys.y', 'b')],
                            outputs=[('subsys.z', 'c')])

p.model.add_subsystem('subprob', subprob, promotes_inputs=['a', 'b'], promotes_outputs=['c'])
p.setup()

p.set_val('a', 1)
p.set_val('b', 2)

p.run_model()

# Example: Using Variables with the Same Name

In [ ]:
import openmdao.api as om

p = om.Problem()

model = om.Group()

model.add_subsystem('x1Comp', om.ExecComp('x1 = x*3'))
model.add_subsystem('x2Comp', om.ExecComp('x2 = x**3'))
model.connect('x1Comp.x1', 'model.x1')
model.connect('x2Comp.x2', 'model.x2')
model.add_subsystem('model', om.ExecComp('z = x1**2 + x2**2'))

subprob = om.SubproblemComp(model=model, inputs=[('x1Comp.x', 'x'), ('x2Comp.x', 'y')],
                            outputs=[('model.z', 'z')])

p.model.add_subsystem('subprob', subprob)
p.setup()

p.set_val('subprob.x', 1)
p.set_val('subprob.y', 2)

p.run_model()